In [87]:
def getpvdata()-> pd.DataFrame:
    #取得http://10.60.59.63:8501/remsweb/apply/rems01022/home 已受理案件資料
    """
    至RNIS取得目前已備案資料
    """
    import requests
    import json
    import pandas as pd
    import xlwings as xw
    url = "http://10.60.59.63:8501/remsapi/apply/rems01022/query"
    payload = {"branchid":"106","energytype":"","status":"","wholesaleType":"","query":"false","export":"false","count":"10"}
    headers = {"Accept":"application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding":"gzip, deflate",
    "Accept-Language":"zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection":"keep-alive",
    "Content-Length":"107",
    "Content-Type":"application/json;charset=UTF-8",
    "Cookie":"JSESSIONID=Krff1T_F8Yq_dXVy-KPFANv4a63SNTQARz6iRPlh-ZABhBFa0Q0C!-1373575639;TS0122fc2b=0101fa365134e6a9664b011c7db74be257d90f02002fd7336305277231539d635bdacd3f83515976fb983bf53931804f28162facd3d14d10f56d6cfae22ab5e868d4ccd523",
    "Host":"10.60.59.63:8501",
    "Nbsdarcd":"08",
    "Nbsuser":"457984",
    "Origin":"http://10.60.59.63:8501",
    "Referer":"http://10.60.59.63:8501/remsweb/apply/rems01022/home",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "X-Requested-With":"XMLHttpRequest"}

    res = requests.post(url,data=json.dumps(payload),headers=headers)
    #res.text 轉為json再轉為dataframe
    res_json = json.loads(res.text)
    df = pd.DataFrame(res_json["data"])
    #drop approvedid 
    df.drop(columns=["approvedid"],inplace=True)
    #Index(['branchid', 'descr', 'approvedid', 'formid', 'applicant', 'srcAddress','deConta', 'wholesaleContain', 'wholesaleDemand', 'contractDate','inpalDate', 'runDate', 'memo'],dtype='object')
    columns = ["區處別","設置型別","受理編號","設置者","設置地點","裝置\n容量","躉售容量","躉售\n方式","簽約日期","併聯日期","運轉中抄日","備註說明"]
    df.columns = columns
    return df
# try:
#     df.to_excel("再生能源-已受理案件.xlsx",index=False)
# except:
#     print("再生能源-已受理案件.xlsx 被開啟")
#     wb = xw.Book("再生能源-已受理案件.xlsx close")
#     #關閉再生能源-已受理案件.xlsx
#     wb.close()
#     df.to_excel("再生能源-已受理案件.xlsx", index=False)

#tpcmeter 台電電號　url =http://10.60.59.63:8501/remsapi/apply/rems01019/query02Part02 method = post payload={"formid":"106100PV0003"}
def tpcmeter(pvnumber:str)->str:
    """
    輸入PV編號取得電號
    """
    import json
    import requests
    headers = {
    "Accept":"application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding":"gzip, deflate",
    "Accept-Language":"zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection":"keep-alive",
    "Content-Length":"25",
    "Content-Type":"application/json;charset=UTF-8",
    "Cookie":"JSESSIONID=qcxUH5FxkgHtPfuGty4ltxxcR95wOEd5IpGwyZTBk5Ozy4FEfylr!-1016557022; TS0122fc2b=0101fa36510d33fabb242a3f3ee227eaf8b92cfa5f4796c2ece4e36122ec24a5f62f29823158068f6492ac17d65db87be54808a6448ee6c36914067d31bc8df06bd2a3a9f7",
    "Host":"10.60.59.63:8501",
    "Nbsdarcd":"08",
    "Nbsuser":"457984",
    "Origin":"http://10.60.59.63:8501",
    "Referer":"http://10.60.59.63:8501/remsweb/apply/rems01019/home",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "X-Requested-With":"XMLHttpRequest"}

    url = "http://10.60.59.63:8501/remsapi/apply/rems01019/query02Part02"

    payloads = json.dumps({"formid":pvnumber})
    res = requests.post(url,headers=headers,data=payloads)
    tpc_meter = json.loads(res.text)["data"][0]["tpcmeter"]
               
    return tpc_meter


#以台電電號 取得座標 http://10.210.35.218/DMQService/api/DistributedCustomerInformation/GetDistributedCustomerInformation
def meternumber_to_tpcdata(meternumber)->tuple:
    """
    回傳三個資料
    1.ownertpclid 台電座標
    2.feeder(codetet) 饋線代號
    3.tpc_address(address) 台電桿號
    """
    #檢查meternumber是否為11碼
    if len(meternumber)<11:
        meternumber = "0"+str(meternumber)

    import requests
    import json
    headers = {
    "Accept":"application/json, text/plain, */*",
    "Accept-Encoding":"gzip, deflate",
    "Accept-Language":"zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection":"keep-alive",
    "Content-Length":"138",
    "Content-Type":"application/json",
    "Host":"10.210.35.218",
    "Origin":"http://10.210.35.218",
    "Referer":"http://10.210.35.218/DMQSWeb/dmqs/distributedcustomer-information",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
    url = "http://10.210.35.218/DMQService/api/DistributedCustomerInformation/GetDistributedCustomerInformation"
    
    payloads = json.dumps({"ohug":"","Codetxt":"","ownertpclid":"","customername":"","meternumber":meternumber,"vlevel":"","hvlv":"","group1":"","paralletype":""})
    res = requests.post(url,headers=headers,data=payloads)
    data = json.loads(res.text)
    #台電座標 ownertpclid
    if data["data"] == []:
        return "", "", ""
    else:
        ownertpclid = data["data"][0]["ownertpclid"]
        #饋線代號
        feeder = data["data"][0]["codetxt"]
        #台電桿號
        tpc_address = data["data"][0]["address"]
        return ownertpclid, feeder, tpc_address 


In [84]:
#電號 這樣跑太久超過22分鐘
df_merged['電號']=df_merged['受理編號'].apply(lambda x: tpcmeter(x))



In [88]:
df_merged[['台電座標','動態饋線','台電桿號']] = df_merged['受理編號'].apply(lambda x: meternumber_to_tpcdata(x))

In [89]:
df_merged.columns

Index([                   '區處別',                   '設置型別',
                         '受理編號',                    '設置者',
                         '設置地點',                 '裝置\n容量',
                         '躉售容量',                 '躉售\n方式',
                         '簽約日期',                   '併聯日期',
                        '運轉中抄日',                   '備註說明',
                       '併聯\n方式',                   '審查人員',
                           '饋線',                     '電號',
       ('台電座標', '動態饋線', '台電桿號')],
      dtype='object')

In [25]:
df = getpvdata()
try:
    df.to_excel("再生能源-已受理案件.xlsx",index=False)
except:
    import xlwings as xw
    print("再生能源-已受理案件.xlsx 被開啟")
    wb = xw.Book("再生能源-已受理案件.xlsx close")
    #關閉再生能源-已受理案件.xlsx
    wb.close()
    df.to_excel("再生能源-已受理案件.xlsx", index=False)

In [70]:
import pandas as pd
pv_path = 'Z:/(靜)-000受理登記簿/PV系統審查登記簿.xlsm'
df1= pd.read_excel(pv_path,sheet_name='113',engine='openpyxl',header=1,usecols='B:E,J,M,T')
df2 = pd.read_excel(pv_path,sheet_name='112',engine='openpyxl',header=1,usecols='B:E,J,M,T')
df3=pd.read_excel(pv_path,sheet_name='111',engine='openpyxl',header=1,usecols='B:E,J,M,T')
df4=pd.read_excel(pv_path,sheet_name='110',engine='openpyxl',header=1,usecols='B:E,J,M,T')
df4.columns = ['受理編號', '設置者', '設置地點', '裝置\n容量 (kW)',  '併聯\n方式','審查人員', '饋線']
df5=pd.read_excel(pv_path,sheet_name='109',engine='openpyxl',header=1,usecols='B:E,H,K,O')
df5.columns = ['受理編號', '設置者', '設置地點', '裝置\n容量 (kW)', '併聯\n方式', '審查人員', '饋線']
df6=pd.read_excel(pv_path,sheet_name='108',engine='openpyxl',header=1,usecols='B:E,G,J,N')
df7=pd.read_excel(pv_path,sheet_name='107',engine='openpyxl',header=1,usecols='B:E,G,J,N')
df7.columns = ['受理編號', '設置者', '設置地點', '裝置\n容量 (kW)', '併聯\n方式', '審查人員', '饋線']
df8=pd.read_excel(pv_path,sheet_name='106',engine='openpyxl',header=1,usecols='B:E,G,J,N')
df8.columns = ['受理編號', '設置者', '設置地點', '裝置\n容量 (kW)', '併聯\n方式', '審查人員', '饋線']
df9=pd.read_excel(pv_path,sheet_name='105',engine='openpyxl',header=1,usecols='B:E,G,J,N')
df9.columns = ['受理編號', '設置者', '設置地點', '裝置\n容量 (kW)', '併聯\n方式', '審查人員', '饋線']
df10=pd.read_excel(pv_path,sheet_name='104',engine='openpyxl',header=1,usecols='B:E,G,J')
df10.columns = ['受理編號', '設置者', '設置地點', '裝置\n容量 (kW)', '併聯\n方式','審查人員']
merged= pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10])



c:\Users\C036039\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\C036039\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\C036039\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\C036039\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [77]:
import pandas as pd
df_merged = pd.merge(df,merged[['受理編號','併聯\n方式','審查人員','饋線']],how='left',on='受理編號')
# 將日期列轉換為 datetime 格式

In [78]:
# 將日期欄位轉換為 datetime 格式
import pandas as pd

# df_merged[['簽約日期','併聯日期','運轉中抄日']]= pd.to_datetime(df_merged[['簽約日期','併聯日期','運轉中抄日']],unit='ms')
df_merged['簽約日期'] = pd.to_datetime(df_merged['簽約日期'], unit='ms').dt.strftime('%Y/%m/%d')
df_merged['併聯日期'] = pd.to_datetime(df_merged['併聯日期'], unit='ms').dt.strftime('%Y/%m/%d')
df_merged['運轉中抄日'] = pd.to_datetime(df_merged['運轉中抄日'], unit='ms').dt.strftime('%Y/%m/%d')

In [85]:
df_merged.to_excel('再生能源-已受理案件.xlsx',index=False)

In [ ]:
import pandas as pd

df= pd.read_excel('再生能源資料-RNIS.xlsx',sheet_name ="案件查詢 - 查詢",usecols='B:W')
df

In [ ]:
import sqlite3
conn = sqlite3.connect('d:/規劃課資料.db')
df.to_sql('再生能源資料', conn, if_exists='replace', index=False)
conn.close()


In [ ]:
df

In [ ]:
#Z:\(靜)-000受理登記簿\PV系統審查登記簿.xlsm 再生能源excel檔, 該檔案為唯讀
#使用pandas做篩選
import pandas as pd
df_b = pd.read_excel(r'Z:\(靜)-000受理登記簿\PV系統審查登記簿.xlsm', sheet_name='113',engine='openpyxl',usecols='B:T')
df_a = pd.read_excel('D:/python/再生能源資料-RNIS.xlsx',sheet_name='案件查詢 - 查詢',usecols='C')
mask =type(df_b.iloc[:,0]) == type(df_a.iloc[:,0])
df_a.iloc[:,0]



In [ ]:
#取得Z:\002再生能源\000-太陽光電\00-審查核准下所有目錄含子目錄的檔案名稱


import os
import pandas as pd

# 指定需要掃描的目錄路徑
base_dir = r'Z:\002再生能源\000-太陽光電\00-審查核准'

# 創建一個空列表存放目錄名稱
directory_names = []

# 遍歷目錄及子目錄
for root, dirs, files in os.walk(base_dir):
    for dir_name in dirs:
        if "內線" in dir_name:
            directory_names.append(os.path.join(root, dir_name))

# 創建一個 DataFrame 並將列表寫入到 Excel 文件中
df = pd.DataFrame({'Directory Name': directory_names})
df.to_excel('filename.xlsx', index=False)


In [ ]:
import os
import pandas as pd

base_dir = r'Z:\\002再生能源\\000-太陽光電\\00-審查核准'
directory_names = []
hyperlinks = []

for root, dirs, files in os.walk(base_dir):
    for dir_name in dirs:
        if "內線" in os.path.basename(dir_name):
            directory_path = os.path.join(root, dir_name)
            directory_names.append(directory_path)
            hyperlinks.append(directory_path)

df = pd.DataFrame({
    'Directory Name': directory_names,
    'hyperlink': hyperlinks
})
df.to_excel('filename.xlsx', index=False, hyperlink=df['hyperlink'])

In [ ]:
import pandas as pd
df = pd.read_excel("filename.xlsx",index=False)
df.to_excel("filename.xlsx",index=False,hyperlinks=df[])

In [ ]:
import pandas as pd

df = pd.read_excel('filename.xlsx')

hyperlink = []
for i, row in df.iterrows():
   
        hyperlinks.append(row[0])

df.to_excel('filename.xlsx', index=False,hyperlinks=hyperlink)

In [3]:
import pandas as pd 
import sqlite3
# Z:/(靜)-000受理登記簿/PV系統審查登記簿.xlsm', sheet_name='113',engine='openpyxl',usecols='B:T')
# d:/python/再生能源資料-RNIS.xlsx
#我想使用類似vlookup方法比較"再生能源資料-RNIS.xlsx 工作表="案件查詢 - 查詢"  c 欄 與"PV系統審查登記簿.xlsm 工作表="113" B欄 若一樣的話將 "PV系統審查登記簿.xlsm" 工作表=113的m欄回貼到 再生能源資料-RNIS.xlsx 的W欄，該怎麼做

rnis_data = pd.read_excel('d:/python/再生能源資料-RNIS.xlsx',sheet_name="案件查詢 - 查詢")
pv_data =pd.read_excel('Z:/(靜)-000受理登記簿/PV系統審查登記簿.xlsm', sheet_name='113',engine='openpyxl',usecols='B:T',header=1)


# 使用merge将两个DataFrame合并，根據"c"列和"B"列进行连接，並將"PV系統審查登記簿.xlsm"工作表=113的"m"列的数据回贴到"再生能源資料-RNIS.xlsx"的"W"列
merged_data = pd.merge(rnis_data, pv_data, left_on='公司編號', right_on='受理編號', how='left')

merged_data.to_excel('d:/python/再生能源資料-RNIS1.xlsx', index=False)

In [18]:
def user_update(mybook="D:/05 - 低頻電驛/每日用戶數.xlsm"):
    """
    mybook = 檔案名稱及路徑 "c:/123/123.xlsx"
    
    存放位置
    工作表名稱   
        
    """


    #用戶數更新    
    import pandas as pd
    import requests 
    import xlwings as xw
    import datetime
    import logging
     
    # 設置日誌檔案名稱和級別
    logging.basicConfig(filename='error.log', level=logging.ERROR)
    wb = xw.Book(mybook)
    today =datetime.datetime.today()
    mytoday =today.strftime("%Y%m%d")
    if mytoday in wb.sheet_names:
        print("已存在")
    else:


        try:
                # 您的程式碼
            

            #用戶更新網址
            url = "http://10.210.35.202/OmsApp201/Users/AllFeederUser.aspx"

            headers = {
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Encoding":"gzip, deflate",
            "Accept-Language":"zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
            "Cache-Control":"max-age=0",
            "Connection":"keep-alive",
            "Content-Length":"287",
            "Content-Type":"application/x-www-form-urlencoded",
            "Cookie":"ASPSESSIONIDSSRDQRDC=AIFKCNCBNLGLKDACJFKJJNOE; ASPSESSIONIDSSQASRDC=LNAEBJPBLJJKDLOFFFDKBLAL; ASP.NET_SessionId=wvo53nrxpobvjtib1pioyobf",
            "Host":"10.210.35.202",
            "Origin":"http://10.210.35.202",
            "Referer":"http://10.210.35.202/OmsApp201/Users/AllFeederUser.aspx",
            "Upgrade-Insecure-Requests":"1",
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"

            }
            payload = ("__VIEWSTATE=%2FwEPDwUJNTY1MTgzNjU5D2QWAgIBD2QWBAIHDzwrAAsAZAIJDw9kFgIeB29uY2xpY2sFD19TaG93TG9hZGluZygpO2Rkg62rXE9%2Br8sYmI7whrKeX3rknxs%3D&__VIEWSTATEGENERATOR=21A69B0D&__EVENTVALIDATION=%2FwEWAwKU%2F4a1AwL8o76uBAKyuOLUC3vnzAZb8Q%2Bf3Vn5rc%2BGcuplCmFq&SubmitButton=%C1%F4%C2%C3%AB%F6%B6s")

            res = requests.post(url,headers=headers,data=payload) 
            df = pd.read_html(res.text,header=0)


            
            formatted_today = today.strftime("%Y-%m-%d")
            formatted_today
            
            sht = wb.sheets["用戶數"]
            sht.range("A1").expand().clear_contents()
            sht["A1"].expand().options(index=None,header=True).value = df[1]
            sht["F1"].value = "饋線更新日期:"
            sht["G1"].value = formatted_today
            #複製用戶數工作表
            sht.copy()
            sht2 = wb.sheets["用戶數 (2)"]
            sht2.name = mytoday
            wb.save()
        except Exception as e:
            # 記錄異常
            logging.error(str(e))  
if __name__ == "__main__":
    user_update()
    # import sys
    # # 將函數名稱映射到函數對象的字典
    # functions = {'user_update': user_update }

    # if len(sys.argv) < 2:
    #     function_name = 'user_update'
    #     mybook = "D:/05 - 低頻電驛/每日用戶數.xlsm"
    # else:
    #     function_name = sys.argv[1]
    #     mybook = sys.argv[2] if len(sys.argv) > 2 else None

    # if function_name in functions:
    #     # 呼叫相應的函數
    #     if mybook:
    #         functions[function_name](mybook)
    #     else:
    #         print(f"請提供 {function_name} 函數的參數")
    # else:
    #     print("未知的函數:", function_name)
    
#執行命令
#C:/Users/C036039/AppData/Local/Programs/Python/Python312/python.exe low_frequency.py user_update "D:\\05 - 低頻電驛\\01-年度檢討
#低頻卸量\\!!!111年低頻動作整理(依1110630填報後更新)_目前最新!!!.xlsx"
# def a(name="john"):
#     print(f"A is {name}")
# def b():
#     print("B")
# functions = {'a': a, 'b': b} 
# import sys


# if len(sys.argv) < 2:
#     print("請提供要執行的函數名稱")
#     sys.exit(1)    
# function_name = sys.argv[1]   
# if function_name in functions:
#         # 呼叫相應的函數
#     functions[function_name](*sys.argv[2:])
# else:
#     print("未知的函數:", function_name)
       

C:\Users\C036039\AppData\Local\Temp\ipykernel_85828\4067999847.py:54: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(res.text,header=0)


In [ ]:
sht.copy()